# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          29 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/28 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.05464


extracting tarball to tmp_2302.05464...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.05465


extracting tarball to tmp_2302.05465...

...

extracting tarball to tmp_2302.06524...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.06530


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.06524/template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2302.06530...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05465-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05465) | **The Demographics, Stellar Populations, and Star Formation Histories of  Fast Radio Burst Host Galaxies: Implications for the Progenitors**  |
|| Alexa C. Gordon, et al. -- incl., <mark>Elizabeth K. Mahoney</mark>, <mark>Kerry Paterson</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *50 pages, 32 figures, 6 tables, submitted*|
|**Abstract**| We present a comprehensive catalog of observations and stellar population properties for 23 highly secure host galaxies of fast radio bursts (FRBs). Our sample comprises six repeating FRBs and 17 apparent non-repeaters. We present 82 new photometric and eight new spectroscopic observations of these hosts. Using stellar population synthesis modeling and employing non-parametric star formation histories (SFHs), we find that FRB hosts have a median stellar mass of $\approx 10^{9.8}\,M_{\odot}$, mass-weighted age of $\approx 5.2$ Gyr, and ongoing star formation rate $\approx 1.3\,M_{\odot}$ yr$^{-1}$ but span wide ranges in all properties. Classifying the hosts by degree of star formation, we find that 91\% (21/23 hosts) are star-forming, one is transitioning, and one is quiescent. The majority trace the star-forming main sequence of galaxies, but at least two FRBs originate in less active environments, both of which are apparent non-repeaters. Across all modeled properties, we find no statistically significant distinction between the hosts of repeaters and non-repeaters. However, the hosts of repeating FRBs generally extend to lower stellar masses, and the hosts of non-repeaters arise in more optically luminous galaxies. Further, the four galaxies with the most clear and prolonged rises in their SFHs all host repeating FRBs, demonstrating heightened star formation activity in the last $\lesssim 100$ Myr. Our results support the young magnetar model for most FRBs in which their progenitors formed through core-collapse supernovae, but the presence of some FRBs in less active environments suggests that a fraction form through more delayed channels. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05694-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05694) | **TOI-2525 b and c: A pair of massive warm giant planets with a strong  transit timing variations revealed by TESS**  |
|| <mark>Trifon Trifonov</mark>, et al. -- incl., <mark>Thomas Henning</mark>, <mark>Melissa J. Hobson</mark>, <mark>Man Hoi Lee</mark>, <mark>Martin Kuerster</mark>, <mark>Rene Heller</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *Accepted for publication in AJ*|
|**Abstract**| TOI-2525 is a K-type star with an estimated mass of M = 0.849$_{-0.033}^{+0.024}$ M$_\odot$ and radius of R = 0.785$_{-0.007}^{+0.007}$ R$_\odot$ observed by the TESS mission in 22 sectors (within sectors 1 and 39). The TESS light curves yield significant transit events of two companions, which show strong transit timing variations (TTVs) with a semi-amplitude of a $\sim$6 hours. We performed TTV dynamical, and photo-dynamical light curve analysis of the TESS data, combined with radial velocity (RV) measurements from FEROS and PFS, and we confirmed the planetary nature of these companions. The TOI-2525 system consists of a transiting pair of planets comparable to Neptune and Jupiter with estimated dynamical masses of $m_{\rm b}$ = 0.088$_{-0.004}^{+0.005}$ M$_{\rm Jup.}$, and $m_{\rm c}$ = 0.709$_{-0.033}^{+0.034}$ M$_{\rm Jup.}$, radius of $r_b$ = 0.88$_{-0.02}^{+0.02}$ R$_{\rm Jup.}$ and $r_c$ = 0.98$_{-0.02}^{+0.02}$ R$_{\rm Jup.}$, and with orbital periods of $P_{\rm b}$ = 23.288$_{-0.002}^{+0.001}$ days and $P_{\rm c}$ = 49.260$_{-0.001}^{+0.001}$ days for the inner and the outer planet, respectively. The period ratio is close to the 2:1 period commensurability, but the dynamical simulations of the system suggest that it is outside the mean motion resonance (MMR) dynamical configuration. TOI-2525 b is among the lowest density Neptune-mass planets known to date, with an estimated median density of $\rho_{\rm b}$ = 0.174$_{-0.015}^{+0.016}$ g\,cm$^{-3}$. The TOI-2525 system is very similar to the other K-dwarf systems discovered by TESS, TOI-2202 and TOI-216, which are composed of almost identical K-dwarf primary and two warm giant planets near the 2:1 MMR. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05464-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05464) | **Galaxy Zoo: Kinematics of strongly and weakly barred galaxies**  |
|| Tobias Géron, et al. -- incl., <mark>Chris Lintott</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *20 pages, 16 figures*|
|**Abstract**| We study the bar pattern speeds and corotation radii of 225 barred galaxies, using IFU data from MaNGA and the Tremaine-Weinberg method. Our sample, which is divided between strongly and weakly barred galaxies identified via Galaxy Zoo, is the largest that this method has been applied to. We find lower pattern speeds for strongly barred galaxies than for weakly barred galaxies. As simulations show that the pattern speed decreases as the bar exchanges angular momentum with its host, these results suggest that strong bars are more evolved than weak bars. Interestingly, the corotation radius is not different between weakly and strongly barred galaxies, despite being proportional to bar length. We also find that the corotation radius is significantly different between quenching and star forming galaxies. Additionally, we find that strongly barred galaxies have significantly lower values for R, the ratio between the corotation radius and the bar radius, than weakly barred galaxies, despite a big overlap in both distributions. This ratio classifies bars into ultrafast bars (R < 1.0; 11% of our sample), fast bars (1.0 < R < 1.4; 27%) and slow bars (R > 1.4; 62%). Simulations show that R is correlated with the bar formation mechanism, so our results suggest that strong bars are more likely to be formed by different mechanisms than weak bars. Finally, we find a lower fraction of ultrafast bars than most other studies, which decreases the recently claimed tension with {\Lambda}CDM. However, the median value of R is still lower than what is predicted by simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05718-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05718) | **Synergies between Venus & Exoplanetary Observations**  |
|| M.J. Way, et al. -- incl., <mark>Helmut Lammer</mark>, <mark>Joseph O'Rourke</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *90 pages, 14 figures, Part of the collection "Venus: Evolution Through Time" this https URL*|
|**Abstract**| In this chapter we examine how our knowledge of present day Venus can inform terrestrial exoplanetary science and how exoplanetary science can inform our study of Venus. In a superficial way the contrasts in knowledge appear stark. We have been looking at Venus for millennia and studying it via telescopic observations for centuries. Spacecraft observations began with Mariner 2 in 1962 when we confirmed that Venus was a hothouse planet, rather than the tropical paradise science fiction pictured. As long as our level of exploration and understanding of Venus remains far below that of Mars, major questions will endure. On the other hand, exoplanetary science has grown leaps and bounds since the discovery of Pegasus 51b in 1995, not too long after the golden years of Venus spacecraft missions came to an end with the Magellan Mission in 1994. Multi-million to billion dollar/euro exoplanet focused spacecraft missions such as JWST, and its successors will be flown in the coming decades. At the same time, excitement about Venus exploration is blooming again with a number of confirmed and proposed missions in the coming decades from India, Russia, Japan, the European Space Agency and the National Aeronautics and Space Administration. In this chapter, we review what is known and what we may discover tomorrow in complementary studies of Venus and its exoplanetary cousins. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05613-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05613) | **KMT-2021-BLG-1122L: The first microlensing triple stellar system**  |
|| Cheongho Han, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Chung-Uk Lee</mark>, <mark>Seung-Lee Kim</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *8 pages, 7 figures, 2 tables*|
|**Abstract**| We systematically inspect the microlensing data acquired by the KMTNet survey during the previous seasons in order to find anomalous lensing events for which the anomalies in the lensing light curves cannot be explained by the usual binary-lens or binary-source interpretations. From the inspection, we find that interpreting the three lensing events OGLE-2018-BLG-0584, KMT-2018-BLG-2119, and KMT-2021-BLG-1122 requires four-body (lens+source) models, in which either both the lens and source are binaries (2L2S event) or the lens is a triple system (3L1S event). Following the analyses of the 2L2S events presented in \citet{Han2023}, here we present the 3L1S analysis of the KMT-2021-BLG-1122. It is found that the lens of the event KMT-2021-BLG-1122 is composed of three masses, in which the projected separations (normalized to the angular Einstein radius) and mass ratios between the lens companions and the primary are $(s_2, q_2)\sim (1.4, 0.53)$ and $(s_3, q_3) \sim (1.6, 0.24)$. By conducting a Bayesian analysis, we estimate that the masses of the individual lens components are $(M_1, M_2, M_3)\sim (0.47\,M_\odot, 0.24\,M_\odot, 0.11\,M_\odot)$. The companions are separated in projection from the primary by $(a_{\perp,2}, a_{\perp,3})\sim (3.5, 4.0)$~AU. The lens of KMT-2018-BLG-2119 is the first triple stellar system detected via microlensing. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05468-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05468) | **Carbonaceous dust grains within galaxies seen in the first billion years  of cosmic time**  |
|| Joris Witstok, et al. -- incl., <mark>Anna de Graaff</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *27 pages, 7 figures. Submitted*|
|**Abstract**| Interstellar dust captures a significant fraction of elements heavier than helium in the solid state and is an indispensable component both in theory and observations of galaxy evolution. Dust emission is generally the primary coolant of the interstellar medium (ISM) and facilitates the gravitational collapse and fragmentation of gas clouds from which stars form, while altering the emission spectrum of galaxies from ultraviolet (UV) to far-infrared wavelengths through the reprocessing of starlight. However, the astrophysical origin of various types of dust grains remains an open question, especially in the early Universe. Here we report direct evidence for the presence of carbonaceous grains from the detection of the broad UV absorption feature around $2175 \, \mathring{\rm A}$ in deep near-infrared spectra of galaxies up to the first billion years of cosmic time, at a redshift ($z$) of $\sim 7$. This dust attenuation feature has previously only been observed spectroscopically in older, more evolved galaxies at redshifts of $z < 3$. The carbonaceous grains giving rise to this feature are often thought to be produced on timescales of hundreds of millions of years by asymptotic giant branch (AGB) stars. Our results suggest a more rapid production scenario, likely in supernova (SN) ejecta. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05920-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05920) | **Three-stage Collapse of the Long Gamma-Ray Burst from GRB 160625B Prompt  Multiwavelength Observations**  |
|| <mark>V. M. Lipunov</mark>, et al. -- incl., <mark>G. V. Lipunova</mark> |
|*Appeared on*| *2023-02-14*|
|*Comments*| *22 pages, 12 figures, 4 tables, accepted for publication in ApJ*|
|**Abstract**| This article presents the early results of synchronous multiwavelength observations of one of the brightest gamma-ray bursts (GRBs) GRB 160625B with the detailed continuous fast optical photometry of its optical counterpart obtained by MASTER and with hard X-ray and gamma-ray emission, obtained by the Lomonosov and Konus-Wind spacecraft. The detailed photometry led us to detect the quasi-periodical emission components in the intrinsic optical emission. As a result of our analysis of synchronous multiwavelength observations, we propose a three-stage collapse scenario for this long and bright GRB. We suggest that quasiperiodic fluctuations may be associated with forced precession of a self-gravitating rapidly rotating superdense body (spinar), whose evolution is determined by a powerful magnetic field. The spinar's mass allows it to collapse into a black hole at the end of evolution. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.05924-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.05924) | **Orphan optical flare as SOSS emission afterglow, localization in time**  |
|| <mark>V. Lipunov</mark>, et al. |
|*Appeared on*| *2023-02-14*|
|*Comments*| *8 pages, 5 figures, accepted for publication in MNRAS*|
|**Abstract**| We report on MASTER optical observations of an afterglow-like optical and X-ray transient AT2021lfa/ZTF21aayokph. We detected the initial steady brightening of the transient at 7{\sigma} confidence level. This allowed us to use smooth optical self-similar emission of GRBs model to constrain the explosion time to better than 14 min as well as to estimate its initial Lorentz factor {\Gamma}0 = 20 +/- 10. Taking into consideration the low {\Gamma}0 and non-detection in gamma-rays, we classify this transient as the first failed GRB afterglow. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.05465.md
    + _build/html/tmp_2302.05465/./figures/luminosity-redshift_updated.jpg
    + _build/html/tmp_2302.05465/./figures/new_spectra_prospector_4x2.jpg
    + _build/html/tmp_2302.05465/./figures/logsSFR_MWA_newanalysis_fullsample_annotated_new.jpg
exported in  _build/html/2302.05694.md
    + _build/html/tmp_2302.05694/./RV_model_sep22.jpg
    + _build/html/tmp_2302.05694/./RV_model_sep22_ph_pl1.jpg
    + _build/html/tmp_2302.05694/./RV_model_sep22_ph_pl2.jpg
    + _build/html/tmp_2302.05694/./out_TOI2525_TTVs_dynNS_SEP_22_4_out_1a.png
    + _build/html/tmp_2302.05694/./RaMa_Teq.jpg
    + _build/html/tmp_2302.05694/./RaMa_pho.jpg


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\NU}{\affiliation{Center for Interdisciplinary Exploration and Research in Astrophysics (CIERA) and Department of Physics and Astronomy, Northwestern University, Evanston, IL 60208, USA}}$
$\newcommand{\Purdue}{\affiliation{Purdue University,$
$Department of Physics and Astronomy, 525 Northwestern Avenue, West Lafayette, IN 47907, USA}}$
$\newcommand{\CfA}{\affiliation{Center for Astrophysics\:|\:Harvard \& Smithsonian, 60 Garden St. Cambridge, MA 02138, USA}}$
$\newcommand{\UCSC}{\affiliation{Department of Astronomy and Astrophysics, University of California, Santa Cruz, CA 95064, USA}}$
$\newcommand{\IS}{\affiliation{Centre for Astrophysics and Cosmology, Science Institute, University of Iceland, Dunhagi 5, 107 Reykjav\'ik, Iceland}}$
$\newcommand{\DAWN}{\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, 2100 Copenhagen \O, Denmark}}$
$\newcommand{\PUCV}{\affiliation{Instituto de F\'isica, Pontificia Universidad Cat\'olica de Valpara\'iso, Casilla 4059, Valpara\'iso, Chile}}$
$\newcommand{\IPMU}{\affiliation{Kavli Institute for the Physics and Mathematics of the Universe (Kavli IPMU), 5-1-5 Kashiwanoha, Kashiwa, 277-8583, Japan}}$
$\newcommand{\PSU}{\affiliation{Department of Astronomy \& Astrophysics, The Pennsylvania State University, University Park, PA 16802, USA}}$
$\newcommand{\ICDS}{\affiliation{Institute for Computational \& Data Sciences, The Pennsylvania State University, University Park, PA, USA}}$
$\newcommand{\IGC}{\affiliation{Institute for Gravitation and the Cosmos, The Pennsylvania State University, University Park, PA 16802, USA}}$
$\newcommand{\Swin}{\affiliation{ Centre for Astrophysics and Supercomputing, Swinburne University of Technology, Hawthorn, VIC, 3122, Australia}}$
$\newcommand{\Curtin}{\affiliation{ International Centre for Radio Astronomy Research, Curtin University, Bentley, WA 6102, Australia}}$
$\newcommand{\MQ}{\affiliation{School of Mathematical and Physical Sciences, Macquarie University, NSW 2109, Australia}}$
$\newcommand{\MQAAAstro}{\affiliation{Macquarie University Research Centre for Astronomy, Astrophysics \& Astrophotonics, Sydney, NSW 2109, Australia}}$
$\newcommand{\CSIRO}{\affiliation{CSIRO, Space and Astronomy, PO Box 76, Epping NSW 1710 Australia}}$
$\newcommand{\KICP}{\affiliation{Kavli Institute for Cosmological Physics, The University of Chicago, 5640 South Ellis Avenue, Chicago, IL 60637, USA}}$
$\newcommand{\UA}{\affiliation{University of Arizona, Steward Observatory, 933~N.~Cherry~Ave., Tucson, AZ 85721, USA}}$
$\newcommand{\EFI}{\affiliation{Enrico Fermi Institute, The University of Chicago, 933 East 56th Street, Chicago, IL 60637, USA}}$
$\newcommand{\mpia}{\affiliation{Max-Planck-Institut f\"{u}r Astronomie (MPIA), K\"{o}nigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\GWU}{\affiliation{Department of Physics, The George Washington University, Washington, DC 20052, USA}}$
$\newcommand{\UCB}{\affiliation{Department of Astronomy, University of California, Berkeley, CA 94720-3411, USA}}$
$\newcommand{\RU}{\affiliation{Department of Astrophysics/IMAPP, Radboud University, PO Box 9010,$
$6500 GL, The Netherlands}}$
$\newcommand{\LJMU}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, IC2, Liverpool Science Park, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{\LU}{\affiliation{School of Physics and Astronomy, University of Leicester, University Road, Leicester. LE1 7RH, UK}}$
$\newcommand{\ARC}{\affiliation{ARC Centre of Excellence for All-Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\ASTRON}{\affiliation{ASTRON, Netherlands Institute for Radio Astronomy, Oude Hoogeveensedijk 4, 7991 PD Dwingeloo, The Netherlands}}$
$\newcommand{\VLBI}{\affiliation{Joint institute for VLBI ERIC, Oude Hoogeveensedijk 4, 7991 PD Dwingeloo, The Netherlands$
$}}$
$\newcommand{\Anton}{\affiliation{Anton Pannekoek Institute for Astronomy, University of Amsterdam, Science Park 904, 1098 XH, Amsterdam, The Netherlands}}$
$\newcommand{\Manchester}{\affiliation{Department of Physics and Astronomy, University of Manchester, Oxford Road, M13 9PL, UK}}$
$\newcommand{\McGill}{\affiliation{Department of Physics, McGill University, Montreal, Quebec H3A 2T8, Canada$
$}}$
$\newcommand{\ICRAR}{\affiliation{International Centre for Radio Astronomy Research (ICRAR), Curtin University, Bentley, WA 6102, Australia}}$
$\newcommand{\SIA}{\affiliation{Sydney Institute for Astronomy, School of Physics A28, University of Sydney, NSW 2006, Australia}}$
$\newcommand{\NAOJ}{\affiliation{Division of Science, National Astronomical Observatory of Japan,2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}}$
$\newcommand{\MPIA}{\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\thefigure}{\arabic{figure} (Cont.)}$
$\newcommand{\thefigure}{\arabic{figure}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\NU$}{\affiliation{Center for Interdisciplinary Exploration and Research in Astrophysics (CIERA) and Department of Physics and Astronomy, Northwestern University, Evanston, IL 60208, USA}}$
$\newcommand{\Purdue}{\affiliation{Purdue University,$
$Department of Physics and Astronomy, 525 Northwestern Avenue, West Lafayette, IN 47907, USA}}$
$\newcommand{$\CfA$}{\affiliation{Center for Astrophysics\:|\:Harvard \& Smithsonian, 60 Garden St. Cambridge, MA 02138, USA}}$
$\newcommand{$\UCSC$}{\affiliation{Department of Astronomy and Astrophysics, University of California, Santa Cruz, CA 95064, USA}}$
$\newcommand{$\IS$}{\affiliation{Centre for Astrophysics and Cosmology, Science Institute, University of Iceland, Dunhagi 5, 107 Reykjav\'ik, Iceland}}$
$\newcommand{$\DAWN$}{\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, 2100 Copenhagen \O, Denmark}}$
$\newcommand{$\PUCV$}{\affiliation{Instituto de F\'isica, Pontificia Universidad Cat\'olica de Valpara\'iso, Casilla 4059, Valpara\'iso, Chile}}$
$\newcommand{$\IPMU$}{\affiliation{Kavli Institute for the Physics and Mathematics of the Universe (Kavli IPMU), 5-1-5 Kashiwanoha, Kashiwa, 277-8583, Japan}}$
$\newcommand{$\PSU$}{\affiliation{Department of Astronomy \& Astrophysics, The Pennsylvania State University, University Park, PA 16802, USA}}$
$\newcommand{$\ICDS$}{\affiliation{Institute for Computational \& Data Sciences, The Pennsylvania State University, University Park, PA, USA}}$
$\newcommand{$\IGC$}{\affiliation{Institute for Gravitation and the Cosmos, The Pennsylvania State University, University Park, PA 16802, USA}}$
$\newcommand{$\Swin$}{\affiliation{ Centre for Astrophysics and Supercomputing, Swinburne University of Technology, Hawthorn, VIC, 3122, Australia}}$
$\newcommand{$\Curtin$}{\affiliation{ International Centre for Radio Astronomy Research, Curtin University, Bentley, WA 6102, Australia}}$
$\newcommand{$\MQ$}{\affiliation{School of Mathematical and Physical Sciences, Macquarie University, NSW 2109, Australia}}$
$\newcommand{$\MQ$AAAstro}{\affiliation{Macquarie University Research Centre for Astronomy, Astrophysics \& Astrophotonics, Sydney, NSW 2109, Australia}}$
$\newcommand{$\CSIRO$}{\affiliation{CSIRO, Space and Astronomy, PO Box 76, Epping NSW 1710 Australia}}$
$\newcommand{$\KICP$}{\affiliation{Kavli Institute for Cosmological Physics, The University of Chicago, 5640 South Ellis Avenue, Chicago, IL 60637, USA}}$
$\newcommand{$\UA$}{\affiliation{University of Arizona, Steward Observatory, 933~N.~Cherry~Ave., Tucson, AZ 85721, USA}}$
$\newcommand{$\EFI$}{\affiliation{Enrico Fermi Institute, The University of Chicago, 933 East 56th Street, Chicago, IL 60637, USA}}$
$\newcommand{$\mpia$}{\affiliation{Max-Planck-Institut f\"{u}r Astronomie (MPIA), K\"{o}nigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{$\GWU$}{\affiliation{Department of Physics, The George Washington University, Washington, DC 20052, USA}}$
$\newcommand{$\UCB$}{\affiliation{Department of Astronomy, University of California, Berkeley, CA 94720-3411, USA}}$
$\newcommand{\RU}{\affiliation{Department of Astrophysics/IMAPP, Radboud University, PO Box 9010,$
$6500 GL, The Netherlands}}$
$\newcommand{$\LJMU$}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, IC2, Liverpool Science Park, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{$\LU$}{\affiliation{School of Physics and Astronomy, University of Leicester, University Road, Leicester. LE1 7RH, UK}}$
$\newcommand{$\ARC$}{\affiliation{ARC Centre of Excellence for All-Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{$\ASTRON$}{\affiliation{ASTRON, Netherlands Institute for Radio Astronomy, Oude Hoogeveensedijk 4, 7991 PD Dwingeloo, The Netherlands}}$
$\newcommand{\VLBI}{\affiliation{Joint institute for VLBI ERIC, Oude Hoogeveensedijk 4, 7991 PD Dwingeloo, The Netherlands$
$}}$
$\newcommand{$\Anton$}{\affiliation{Anton Pannekoek Institute for Astronomy, University of Amsterdam, Science Park 904, 1098 XH, Amsterdam, The Netherlands}}$
$\newcommand{$\Manchester$}{\affiliation{Department of Physics and Astronomy, University of Manchester, Oxford Road, M13 9PL, UK}}$
$\newcommand{\McGill}{\affiliation{Department of Physics, McGill University, Montreal, Quebec H3A 2T8, Canada$
$}}$
$\newcommand{$\ICRAR$}{\affiliation{International Centre for Radio Astronomy Research (ICRAR), Curtin University, Bentley, WA 6102, Australia}}$
$\newcommand{$\SIA$}{\affiliation{Sydney Institute for Astronomy, School of Physics A28, University of Sydney, NSW 2006, Australia}}$
$\newcommand{$\NAOJ$}{\affiliation{Division of Science, National Astronomical Observatory of Japan,2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}}$
$\newcommand{$\MPIA$}{\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{$\thefigure$}{\arabic{figure} (Cont.)}$
$\newcommand{$\thefigure$}{\arabic{figure}}$</div>



<div id="title">

# The Demographics, Stellar Populations, and Star Formation Histories of Fast Radio Burst Host Galaxies: Implications for the Progenitors

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.05465-b31b1b.svg)](https://arxiv.org/abs/2302.05465)<mark>Appeared on: 2023-02-14</mark> - _50 pages, 32 figures, 6 tables, submitted_

</div>
<div id="authors">

Alexa C. Gordon, et al. -- incl., <mark>Elizabeth K. Mahoney</mark>, <mark>Kerry Paterson</mark>

</div>
<div id="abstract">

**Abstract:** We present a comprehensive catalog of  observations and stellar population properties for 23 highly secure host galaxies of fast radio bursts (FRBs). Our sample comprises six repeating FRBs and 17 apparent non-repeaters. We present 82 new photometric and eight new spectroscopic observations of these hosts. Using stellar population synthesis modeling and employing non-parametric star formation histories (SFHs), we find that FRB hosts have a median stellar mass of$\approx 10^{9.8} M_{\odot}$, mass-weighted age of$\approx 5.2$Gyr, and ongoing star formation rate$\approx 1.3 M_{\odot}$yr$^{-1}$but span wide ranges in all properties. Classifying the hosts by degree of star formation, we find that 91\%(21/23 hosts) are star-forming, one is transitioning, and one is quiescent. The majority trace the star-forming main sequence of galaxies, but at least two FRBs originate in less active environments, both of which are apparent non-repeaters. Across all modeled properties, we find no statistically significant distinction between the hosts of repeaters and non-repeaters. However, the hosts of repeating FRBs generally extend to lower stellar masses, and the hosts of non-repeaters arise in more optically luminous galaxies. Further, the four galaxies with the most clear and prolonged rises in their SFHs all host repeating FRBs, demonstrating heightened star formation activity in the last$\lesssim 100$Myr. Our results support the young magnetar model for most FRBs in which their progenitors formed through core-collapse supernovae, but the presence of some FRBs in less active environments suggests that a fraction form through more delayed channels.

</div>

<div id="div_fig1">

<img src="tmp_2302.05465/./figures/luminosity-redshift_updated.jpg" alt="Fig1" width="100%"/>

**Figure 1. -** Luminosity--redshift distribution for all FRB hosts. We denote repeaters by blue stars and non-repeaters by pink squares. We also include FRB hosts from the literature that did not meet our sample criteria as open symbols. (*fig:l-z*)

</div>
<div id="div_fig2">

<img src="tmp_2302.05465/./figures/new_spectra_prospector_4x2.jpg" alt="Fig4" width="100%"/>

**Figure 4. -** New spectroscopic observations included in this work from SOAR/Goodman, Keck/DEIMOS, and Keck/LRIS. Major emission and absorption lines are denoted by colored lines: Balmer lines in green, oxygen lines in blue, nitrogen lines in orange, sulphur lines in purple, and calcium lines in pink. The spectra are normalized to the photometry in \texttt{Prospector}. The \texttt{Prospector}-derived redshifts are listed in each panel along with the facility/instrument of observation. (*fig:new spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2302.05465/./figures/logsSFR_MWA_newanalysis_fullsample_annotated_new.jpg" alt="Fig9" width="100%"/>

**Figure 9. -** log(${\rm sSFR}_{\rm 0-100 Myr}$) vs. $t_{\rm m}$ of FRB host galaxies compared to COSMOS field galaxies for three redshift bins. The parameter space is divided into Star-Forming, Transitioning, and Quiescent following the \citet{Tacchella+22} classification. Repeaters are denoted by stars and non-repeaters by squares. Both are color-coded by their classification type. Error bars correspond to 68\% confidence. (*fig:sSFR-MWA*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\tess}{\emph{TESS}}$
$\newcommand{\autoref}$
$\newcommand{\thetable}{A\arabic{table}}$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\figureautorefname}{Fig.}$
$\newcommand{\sectionautorefname}{Sect.}$
$\newcommand{\subsectionautorefname}{Sect.}$
$\newcommand{\subsubsectionautorefname}{Sect.}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\tess$}{\emph{TESS}}$
$\newcommand{$\autoref$}$
$\newcommand{$\thetable$}{A\arabic{table}}$
$\newcommand{$\thefigure$}{A\arabic{figure}}$
$\newcommand{$\equationautorefname$}{Eq.}$
$\newcommand{$\figureautorefname$}{Fig.}$
$\newcommand{$\sectionautorefname$}{Sect.}$
$\newcommand{$\subsectionautorefname$}{Sect.}$
$\newcommand{$\subsubsectionautorefname$}{Sect.}$</div>



<div id="title">

# Based on observations collected at the European Organization for Astronomical Research in the Southern Hemisphere under MPG programmes 0104.A-9007. This paper includes data gathered with the 6.5 meter Magellan Telescopes located at Las Campanas Observatory, Chile.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.05694-b31b1b.svg)](https://arxiv.org/abs/2302.05694)<mark>Appeared on: 2023-02-14</mark> - _Accepted for publication in AJ_

</div>
<div id="authors">

<mark>Trifon Trifonov</mark>, et al. -- incl., <mark>Thomas Henning</mark>, <mark>Melissa J. Hobson</mark>, <mark>Man Hoi Lee</mark>, <mark>Ren{\'e} Heller</mark>

</div>
<div id="abstract">

**Abstract:** TOI-2525 is a K-type star with an estimated mass of M = 0.849$_{-0.033}^{+0.024}$M$_\odot$and radius ofR = 0.785$_{-0.007}^{+0.007}$R$_\odot$observed by the TESS mission in 22 sectors (within sectors 1 and 39).The TESS light curves yield significant transit events of two companions, whichshow strong transit timing variations (TTVs) with a semi-amplitude of a$\sim$6 hours.We performed TTV dynamical, and photo-dynamical light curve analysis of the TESS data, combined with radial velocity (RV) measurements from FEROS and PFS, and we confirmed the planetary nature of these companions.The TOI-2525 system consists of a transiting pair of planets comparable to Neptune and Jupiter withestimated dynamical masses of$m_{\rm b}$= 0.088$_{-0.004}^{+0.005}$M$_{\rm Jup.}$,and$m_{\rm c}$= 0.709$_{-0.033}^{+0.034}$M$_{\rm Jup.}$, radius of$r_b$= 0.88$_{-0.02}^{+0.02}$R$_{\rm Jup.}$and$r_c$= 0.98$_{-0.02}^{+0.02}$R$_{\rm Jup.}$, and with orbital periods of$P_{\rm b}$=  23.288$_{-0.002}^{+0.001}$days and$P_{\rm c}$= 49.260$_{-0.001}^{+0.001}$days for the inner and the outer planet, respectively. The period ratio is close tothe 2:1 period commensurability, but the dynamical simulations of the system suggestthat it is outside the mean motion resonance (MMR) dynamical configuration. TOI-2525 b is among the lowest density Neptune-mass planets known to date, with an estimated median density of$\rho_{\rm b}$= 0.174$_{-0.015}^{+0.016}$g cm$^{-3}$.The TOI-2525 system is very similar to the other K-dwarf systems discovered by TESS, TOI-2202 and TOI-216, which are composed of almost identical K-dwarf primary and two warm giant planets near the 2:1 MMR.

</div>

<div id="div_fig1">

<img src="tmp_2302.05694/./RV_model_sep22.jpg" alt="Fig10.1" width="33%"/><img src="tmp_2302.05694/./RV_model_sep22_ph_pl1.jpg" alt="Fig10.2" width="33%"/><img src="tmp_2302.05694/./RV_model_sep22_ph_pl2.jpg" alt="Fig10.3" width="33%"/>

**Figure 10. -** 
    RV component of the two-planet dynamical model of TOI-2525, constructed jointly with the TTVs data and with the RVs from PFS (blue circles). The two RVs from FEROS (red triangles) are overploted with an optimized RV offset to the model. The left panel shows the time series
    and the N-body model,
    the middle and right panels show a phase-folded representation of the RV data, modeled with
    the dynamical model (with an osculating period). The PFS data uncertainties include the estimated RV jitter, added in quadrature to the error budget, while the FEROS data are plotted with their nominal RV uncertainties. The small sub-panels show the RVs residuals.
 (*RV_plot1*)

</div>
<div id="div_fig2">

<img src="tmp_2302.05694/./out_TOI2525_TTVs_dynNS_SEP_22_4_out_1a.png" alt="Fig11" width="100%"/>

**Figure 11. -** Posteriors of the dynamical properties at the 2:1 period ratio commensurability of the two-planet system TOI-2525. The posteriors are generated by randomly drawing 10 000 samples from the TTV+RV dynamical model.
Each sample is tested for stability and the overall dynamical properties are evaluated at the 2:1 period ratio commensurability. The derived dynamical parameters are: mean period ratio $\hat{P}_{\rm rat.}$, mean eccentricities $\hat{e}_{\rm b}$, $\hat{e}_{\rm c}$, mean orbital aligment $\hat{\Delta\omega}$, and their peak-to-peak amplitudes Ampl. $e_{\rm b}$, Ampl. $e_{\rm c}$, Ampl. $\Delta\omega$, and the planetary dynamical masses and semi-major axes. Note that the posterior distribution of $\theta_1$, and $\theta_2$ are not shown, since these exhibit circulation between 0$^\circ$ and 360$^\circ$.
The black contours on the 2D panels represent the 1, 2 and 3$\sigma$ confidence level of the overall posterior samples.
 (*dyn_samp*)

</div>
<div id="div_fig3">

<img src="tmp_2302.05694/./RaMa_Teq.jpg" alt="Fig14.1" width="50%"/><img src="tmp_2302.05694/./RaMa_pho.jpg" alt="Fig14.2" width="50%"/>

**Figure 14. -** Radius-mass demographics for all giant transiting planets with measured
masses by TTVs or RVs. The position of TOI-2525 b and c are plotted with "star" symbols. Panel a) shows the mass-radius relation color-coded by T$_{\rm eq.}$. Panel b) shows the same as panel a), but is color-coded with the planetary density.
The solid curve corresponds to the predicted radius using the models of \citet{Fortney2007} given the estimated luminosity of TOI-2525, a semi-major axis=0.2 AU, and an age of 3.1 G yr.\looseness=-5
 (*hhe_fraction*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

201  publications in the last 7 days.
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers